In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
%pip install langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 1.9 MB/s eta 0:00:02
   ----------------- ---------------------- 1.0/2.4 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.4 MB 1.8 MB/s eta 0:00:01
   ------------------------------ --------- 1.8/2.4 MB 1.8 MB/s eta 0:00:01
   ---------------------------------- ----- 2.1/2.4 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 1.8 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
   ------

In [4]:
from langchain.chat_models import ChatOpenAI

In [13]:
from dotenv import load_dotenv

load_dotenv() # take environment variables form .env


True

In [14]:
KEY = os.getenv('OPENAI_API_KEY')

In [15]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = 'gpt-4o-mini', temperature = 0.5)

In [16]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [18]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables= ['text','number', 'subject', 'tone','response_json'],
    template = TEMPLATE
)

In [26]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = 'quiz', verbose = True)

In [27]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [28]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables= ['subject', 'quiz'],
    template=TEMPLATE2
)

In [29]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = 'review', verbose = True)

In [31]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain, review_chain],
    input_variables = ['text','number', 'subject', 'tone','response_json'],
    output_variables = ['quiz', 'review'],
    verbose = True
)

In [34]:
file_path = r"C:\Users\munnu\OneDrive\Documents\GenAI\MCQGen\data.txt"

In [35]:
file_path

'C:\\Users\\munnu\\OneDrive\\Documents\\GenAI\\MCQGen\\data.txt'

In [36]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [37]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [38]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [41]:
NUMBER = 5
SUBJECT = 'Biology'
TONE = 'simple'

In [42]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            'text': TEXT,
            'number':NUMBER,
            'subject': SUBJECT,
            'tone': TONE,
            'response_json': json.dumps(RESPONSE_JSON)

        }
    )

C:\Users\munnu\AppData\Local\Temp\ipykernel_18440\2967716633.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [43]:
print(f'Total tokens: {cb.total_tokens}')
print(f'Prompt tokens: {cb.prompt_tokens}')
print(f'Completion Tokens: {cb.completion_tokens}')
print(f'Total Cost: {cb.total_cost}')

Total tokens: 1715
Prompt tokens: 983
Completion Tokens: 732
Total Cost: 0.00058665


In [46]:
quiz= response.get('quiz')

In [52]:
quiz = json.loads(quiz)

In [54]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options = '|'.join(
        [
            f'{option} : {option_value}'
            for option , option_value in value['options'].items()
    
        ]
    )
    correct = value['correct']
    quiz_table_data.append({'MCQ' : mcq, 'Choices' : options, 'Correct' : correct})

In [55]:
quiz_table_data

[{'MCQ': 'What is biology the scientific study of?',
  'Choices': 'a : Rocks|b : Life|c : Weather|d : Space',
  'Correct': 'b'},
 {'MCQ': 'What major theme explains the unity and diversity of life?',
  'Choices': 'a : Cell structure|b : Energy processing|c : Evolution|d : Genetics',
  'Correct': 'c'},
 {'MCQ': 'What do all organisms use to regulate their internal environments?',
  'Choices': 'a : Energy|b : Cells|c : Nutrients|d : Homeostasis',
  'Correct': 'd'},
 {'MCQ': 'Which of the following is a type of organism studied in biology?',
  'Choices': 'a : Planets|b : Fungi|c : Rivers|d : Clouds',
  'Correct': 'b'},
 {'MCQ': 'How long ago did life on Earth emerge?',
  'Choices': 'a : 1 billion years|b : 3.7 billion years|c : 10 million years|d : 500 million years',
  'Correct': 'b'}]

In [58]:
quiz = pd.DataFrame(quiz_table_data)

In [59]:
quiz.to_csv('biology-quiz.csv', index = False)